In [1]:
import pandas as pd
import os

def get_image_path(image_id:int):
    return os.path.join('../tiles_768', str(image_id))

I_FOLD = 3
train = pd.read_csv(f"train_fold_{I_FOLD}.csv")
validation = pd.read_csv(f"val_fold_{I_FOLD}.csv")

train['tile_path'] = train['image_id'].apply(lambda x: get_image_path(x))
validation['tile_path'] = validation['image_id'].apply(lambda x: get_image_path(x))
train.head()

,image_id,label,image_width,image_height,is_tma,tile_path
0,4,HGSC,23785,20008,False,../tiles_768/4
1,66,LGSC,48871,48195,False,../tiles_768/66
2,91,HGSC,3388,3388,True,../tiles_768/91
3,281,LGSC,42309,15545,False,../tiles_768/281
4,286,EC,37204,30020,False,../tiles_768/286


In [2]:
# """
# FOR FOLD_3 I USE DROP PATH RATE = 0.5
# """

# import json

# # Open the JSON file for reading
# with open('./convnextv2_large_config/config.json', 'r') as file:
#     data = json.load(file)
    
# data['drop_path_rate'] = 0.5

# with open('./convnextv2_large_config/config.json', 'w') as file:
#     json.dump(data, file, indent=4)

In [2]:
from PIL import Image
import torch
import torch.nn as nn
from transformers import ConvNextV2ForImageClassification
import copy

device = "cuda" if torch.cuda.is_available() else "cpu"
# model_name = "facebook/convnextv2-large-22k-384"
# print(f"Using device {device} and model {model_name}")

model = ConvNextV2ForImageClassification.from_pretrained('./convnextv2_large_config')

model.classifier = nn.Linear(model.classifier.in_features, 5)

state_dict = torch.load(f'convnextv2_large_models/fold_{I_FOLD}/final.pth', map_location=device)
model.load_state_dict(state_dict, strict=False)

model = model.to(device)

# Initialize EMA model
ema_decay = 0.999  # decay factor for EMA
ema_model = copy.deepcopy(model)
ema_model = ema_model.to(device)

NameError: name 'model' is not defined

In [3]:
import os
from PIL import Image
from torch.utils.data import Dataset

integer_to_label = {
    0: 'HGSC',
    1: 'CC',
    2: 'EC',
    3: 'LGSC',
    4: 'MC',
}

label_to_integer = {
    'HGSC': 0,
    'CC': 1,
    'EC': 2,
    'LGSC': 3,
    'MC': 4,
}

class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.folder_paths = []
        self.labels = []
        self.image_ids = []
        for index, row in dataframe.iterrows():
            folder_path = row['tile_path']
            label = row['label']
            image_id = row['image_id']
            if os.path.isdir(folder_path):  # Check if the folder_path is a valid directory
                self.folder_paths.append(folder_path)
                self.labels.append(label)
                self.image_ids.append(image_id)

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_paths = os.listdir(self.folder_paths[idx])
        image_index = random.randint(0, len(image_paths) - 1)
        while not image_paths[image_index].lower().endswith('.png'):  # Check if the file is a PNG
            image_index = random.randint(0, len(image_paths) - 1)

        image = Image.open(os.path.join(self.folder_paths[idx], image_paths[image_index]))
        label = self.labels[idx]
        image_id = self.image_ids[idx]
        if self.transform:
            image = self.transform(image)
        return image, label_to_integer[label], image_id


In [4]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

BATCH_SIZE = 8

"""
I SHOULD BE USING THESE MEANS AND STDS INSTEAD -- NOTE THIS IN THE FUTURE
"image_mean": [
    0.485,
    0.456,
    0.406
],
"image_std": [
    0.229,
    0.224,
    0.225
],
"""

train_transform = transforms.Compose([
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomAffine(degrees=45, translate=(0.25, 0.25), scale=(1, 2), shear=(-30, 30, -30, 30)),
    transforms.Resize(384),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
])

val_transform = transforms.Compose([
    transforms.Resize(384),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

train_dataset = ImageDataset(dataframe=train, transform=train_transform)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=3, shuffle=True)

In [6]:
import logging
import sys

# Get the root logger
logger = logging.getLogger()

# Optional: Remove all existing handlers from the logger
for handler in logger.handlers[:]:
    logger.removeHandler(handler)

# Set the logging level
logger.setLevel(logging.INFO)

# Create a FileHandler and add it to the logger
file_handler = logging.FileHandler(f'logs/convnextv2_large/fold_{I_FOLD}.txt')
file_handler.setLevel(logging.INFO)
logger.addHandler(file_handler)

# Create a StreamHandler for stderr and add it to the logger
stream_handler = logging.StreamHandler(sys.stderr)
stream_handler.setLevel(logging.ERROR)  # Only log ERROR and CRITICAL messages to stderr
logger.addHandler(stream_handler)

In [7]:
train.groupby('label').count()

,image_id,image_width,image_height,is_tma,tile_path
label,,,,,
CC,79,79,79,79,79
EC,100,100,100,100,100
HGSC,178,178,178,178,178
LGSC,37,37,37,37,37
MC,37,37,37,37,37


In [8]:
import torch
import torch.optim as optim
import logging
import numpy as np
import math
from sklearn.metrics import balanced_accuracy_score
import random
from torch.cuda.amp import GradScaler, autocast

initial_lr = 1e-5
final_lr = 1e-6
num_epochs = 10000

# Function for linear warmup
def learning_rate(step, warmup_steps=500, max_steps=20000):
    if step < warmup_steps:
        return initial_lr * (float(step) / float(max(1, warmup_steps)))
    elif step < max_steps:
        progress = (float(step - warmup_steps) / float(max(1, max_steps - warmup_steps)))
        cos_component = 0.5 * (1 + math.cos(math.pi * progress))
        return final_lr + (initial_lr - final_lr) * cos_component
    else:
        return final_lr

# Function to calculate weighted accuracy
def weighted_accuracy(true_labels, predictions, class_weights):
    correct = 0
    total_weight = 0

    for label, pred in zip(true_labels, predictions):
        if label == pred:
            correct += class_weights[label]
        total_weight += class_weights[label]

    return correct / total_weight

def update_ema_variables(model, ema_model, alpha, global_step):
    # Update the EMA model parameters
    with torch.no_grad():
        for ema_param, param in zip(ema_model.parameters(), model.parameters()):
            ema_param.data.mul_(alpha).add_(param.data, alpha=1 - alpha)

scaler = GradScaler()
            
# Define the optimizer
optimizer = optim.AdamW(model.parameters(), lr=initial_lr, weight_decay=5e-2)

# Calculate class weights
class_counts = np.array([178, 79, 100, 37, 37], dtype=np.float32)
class_weights = 1. / class_counts
class_weights /= class_weights.sum()

# Convert class weights to tensor
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Define the loss function with class weights
criterion = torch.nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1)

best_val_accuracy = 0.0
step = 0

for epoch in range(num_epochs):
    model.train()  # set the model to training mode
    
    for i, (images, labels, _) in enumerate(train_dataloader, 0):
        # Convert images to PIL format
        images = images.to(device)
        labels = labels.to(device)
        
        # Linearly increase the learning rate
        lr = learning_rate(step)
        for g in optimizer.param_groups:
            g['lr'] = lr
            
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass with autocast
        with autocast():
            outputs = model(images)
            logits_per_image = outputs.logits
            loss = criterion(logits_per_image, labels)
        
        # Backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        update_ema_variables(model, ema_model, ema_decay, step)

        logging.info('[%d, %5d] loss: %.3f' % (epoch + 1, step, loss.item()))

        if step % 500 == 0:
            ema_model.eval()

            tma_preds = []
            tma_labels = []
            
            non_tma_preds = []
            non_tma_labels = []

            with torch.no_grad():
                for _, row in validation.iterrows():
                    path = row['tile_path']
                    all_files = [f for f in os.listdir(path) if f.lower().endswith('.png')]
                    
                    probabilities = torch.zeros(5).to(device)

                    # Prepare a list to hold image tiles
                    batch_tiles = []

                    sample_size = min(16, len(all_files))
                    sampled_files = random.sample(all_files, sample_size)

                    for image_name in sampled_files:
                        image_path = os.path.join(path, image_name)
                        sub_image = Image.open(image_path)

                        tile = val_transform(sub_image).unsqueeze(0).to(device)
                        batch_tiles.append(tile)

                    outputs = ema_model(torch.concat(batch_tiles, dim=0))
                    probs = outputs.logits.softmax(dim=1)
                    probabilities += probs[0]

                    pred_label = integer_to_label[probabilities.argmax().detach().cpu().item()]
                    label = row['label']
                    if row['is_tma']:
                        tma_preds.append(pred_label)
                        tma_labels.append(label)
                    else:
                        non_tma_preds.append(pred_label)
                        non_tma_labels.append(label)

            tma_accuracy = balanced_accuracy_score(tma_labels, tma_preds)
            non_tma_accuracy = balanced_accuracy_score(non_tma_labels, non_tma_preds)
            accuracy = (tma_accuracy + non_tma_accuracy) / 2
            logging.info(f'TMA Accuracy: {tma_accuracy} | Non-TMA Accuracy: {non_tma_accuracy} | Overall Accuracy: {accuracy}')

            if accuracy > best_val_accuracy:
                best_val_accuracy = accuracy
                torch.save(ema_model.state_dict(), f'convnextv2_large_models/fold_{I_FOLD}/epoch_{epoch}_step_{step}.pth')
                logging.info(f'Model saved after epoch {epoch} and step {step}')
                
            model.train()

        if step == 20000:
            torch.save(ema_model.state_dict(), f'convnextv2_large_models/fold_{I_FOLD}/final.pth')

        step += 1


KeyboardInterrupt



In [7]:
import random
from sklearn.metrics import balanced_accuracy_score

ema_model.eval()

image_ids = []
logits = []
labels = []

with torch.no_grad():
    for _, row in validation.iterrows():
        path = row['tile_path']
        all_files = [f for f in os.listdir(path) if f.lower().endswith('.png')]

        sum_probabilities = torch.zeros(5).to(device)
        sum_log_probabilities = torch.zeros(5).to(device)
        sum_log_neg_probabilities = torch.zeros(5).to(device)
        
        batch_logits = []

        # Prepare a list to hold image tiles
        batch_tiles = []

        sample_size = min(128, len(all_files))
        sampled_files = random.sample(all_files, sample_size)

        for image_name in sampled_files:
            image_path = os.path.join(path, image_name)
            sub_image = Image.open(image_path)

            tile = val_transform(sub_image).unsqueeze(0).to(device)
            batch_tiles.append(tile)
        
        for i_batch in range(0, len(batch_tiles), 32):
            outputs = ema_model(torch.concat(batch_tiles[i_batch:i_batch+32], dim=0))
            probs = outputs.logits.softmax(dim=1)
            batch_logits.append(outputs.logits)
            sum_probabilities += probs.sum(dim=0)
            sum_log_probabilities += torch.log(probs).sum(dim=0)
            sum_log_neg_probabilities += torch.log(1 - probs).sum(dim=0)
        
        image_id = row['image_id']
        batch_logits = torch.concat(batch_logits, dim=0)
        label = row['label']
        print(batch_logits.shape, label, image_id)
        image_ids.append(image_id)
        logits.append(batch_logits)
        labels.append(label)
        

# tma_accuracy = balanced_accuracy_score(tma_labels, tma_preds)


torch.Size([128, 5]) LGSC 33976
torch.Size([128, 5]) LGSC 34277
torch.Size([128, 5]) LGSC 34688
torch.Size([16, 5]) MC 35565
torch.Size([15, 5]) LGSC 37385
torch.Size([128, 5]) HGSC 37655
torch.Size([128, 5]) HGSC 38018
torch.Size([128, 5]) HGSC 38041
torch.Size([128, 5]) HGSC 38048
torch.Size([128, 5]) HGSC 38097
torch.Size([128, 5]) LGSC 38366
torch.Size([128, 5]) HGSC 38535
torch.Size([128, 5]) LGSC 38585
torch.Size([128, 5]) LGSC 38849
torch.Size([128, 5]) LGSC 38959
torch.Size([128, 5]) HGSC 39144
torch.Size([128, 5]) HGSC 39146
torch.Size([128, 5]) HGSC 39172
torch.Size([128, 5]) HGSC 39255
torch.Size([128, 5]) LGSC 39258
torch.Size([128, 5]) HGSC 39297
torch.Size([128, 5]) HGSC 39365
torch.Size([128, 5]) HGSC 39425
torch.Size([128, 5]) LGSC 39466
torch.Size([128, 5]) HGSC 39728
torch.Size([128, 5]) MC 39872
torch.Size([128, 5]) HGSC 39893
torch.Size([128, 5]) HGSC 39990
torch.Size([128, 5]) CC 40129
torch.Size([128, 5]) HGSC 40639
torch.Size([128, 5]) CC 40888
torch.Size([128, 5

In [13]:
predictions = []
for image_logits in logits:
    argmax_indices = torch.argmax(image_logits, dim=1)

    frequency_counts = torch.bincount(argmax_indices, minlength=5)

    max_vote_key = frequency_counts.argmax().cpu().item()
    predictions.append(integer_to_label[max_vote_key])

plurality_accuracy = balanced_accuracy_score(labels, predictions)
plurality_accuracy

0.8529292929292929

In [14]:
predictions = []
for image_logits in logits:
    summed_logits = image_logits.sum(dim=0)
    
    max_vote_key = summed_logits.argmax().cpu().item()
    predictions.append(integer_to_label[max_vote_key])

logit_sum_accuracy = balanced_accuracy_score(labels, predictions)
logit_sum_accuracy

0.8667171717171718

In [15]:
predictions = []
for image_logits in logits:
    summed_probs = image_logits.softmax(dim=1).sum(dim=0)
    
    max_vote_key = summed_probs.argmax().cpu().item()
    predictions.append(integer_to_label[max_vote_key])

prob_sum_accuracy = balanced_accuracy_score(labels, predictions)
prob_sum_accuracy

0.8667171717171718

In [16]:
predictions = []
for image_logits in logits:
    summed_log_probs = torch.log(image_logits.softmax(dim=1)).sum(dim=0)
    
    max_vote_key = summed_log_probs.argmax().cpu().item()
    predictions.append(integer_to_label[max_vote_key])

log_prob_sum_accuracy = balanced_accuracy_score(labels, predictions)
log_prob_sum_accuracy

0.8667171717171718

In [17]:
predictions = []
for image_logits in logits:
    summed_one_minus_log_probs = torch.log(1 - image_logits.softmax(dim=1)).sum(dim=0)
    
    min_vote_key = summed_one_minus_log_probs.argmin().cpu().item()
    predictions.append(integer_to_label[min_vote_key])

one_minus_log_prob_sum_accuracy = balanced_accuracy_score(labels, predictions)
one_minus_log_prob_sum_accuracy

0.8667171717171718

In [30]:
from collections import defaultdict

def borda_count_winner(image_logits):
    num_classes = image_logits.size(1)
    borda_scores = defaultdict(int)

    # Rank each class for each image_logits and assign Borda points
    for logits in image_logits:
        # Get ranks (in descending order of logits)
        ranks = torch.argsort(logits, descending=True)

        # Assign Borda points (highest rank gets num_classes - 1 points, next gets num_classes - 2, ...)
        for rank, class_index in enumerate(ranks):
            borda_scores[class_index.item()] += num_classes - 1 - rank

    # Find the class with the highest total Borda score
    borda_winner = max(borda_scores, key=borda_scores.get)
    return borda_winner

predictions = []
for image_logits in logits:
    borda_winner = borda_count_winner(image_logits)
    predictions.append(integer_to_label[borda_winner])
    
borda_accuracy = balanced_accuracy_score(labels, predictions)
borda_accuracy

0.862929292929293

In [31]:
def instant_runoff_winner(image_logits):
    num_voters, num_classes = image_logits.shape
    active_candidates = set(range(num_classes))

    while True:
        # Count the first-preference votes for each candidate
        first_pref_counts = torch.zeros(num_classes)
        for logits in image_logits:
            for rank in torch.argsort(logits, descending=True):
                if rank.item() in active_candidates:
                    first_pref_counts[rank] += 1
                    break

        # Check if any candidate has more than 50% of the votes
        if torch.any(first_pref_counts > num_voters / 2):
            return torch.argmax(first_pref_counts).item()

        # Eliminate the candidate with the fewest votes
        min_votes, min_candidate = torch.min(first_pref_counts[list(active_candidates)], 0)
        active_candidates.remove(min_candidate.item())
        
predictions = []
for image_logits in logits:
    irv_winner = instant_runoff_winner(image_logits)
    predictions.append(integer_to_label[irv_winner])
    
instant_runoff_accuracy = balanced_accuracy_score(labels, predictions)
instant_runoff_accuracy

0.8574747474747475

In [33]:
def calculate_minimax_winner(image_logits):
    num_voters, num_classes = image_logits.shape
    max_regrets = torch.zeros(num_classes)

    # Perform pairwise comparisons between all classes
    for i in range(num_classes):
        for j in range(i + 1, num_classes):
            # Count how many voters prefer class i over class j and vice versa
            votes_for_i = torch.sum(image_logits[:, i] > image_logits[:, j])
            votes_for_j = num_voters - votes_for_i

            # Update the maximum regret for each class
            max_regrets[i] = max(max_regrets[i], votes_for_j)
            max_regrets[j] = max(max_regrets[j], votes_for_i)

    # The winner is the class with the smallest maximum regret
    return torch.argmin(max_regrets).item()
        
predictions = []
for image_logits in logits:
    minimax_winner = calculate_minimax_winner(image_logits)
    predictions.append(integer_to_label[minimax_winner])
    
minimax_accuracy = balanced_accuracy_score(labels, predictions)
minimax_accuracy

0.8529292929292929

In [38]:
from collections import defaultdict

def find_winner(graph, source, visited):
    """Helper function to find the winner in the graph."""
    if source not in graph:
        return False

    visited.add(source)
    for target in graph[source]:
        if target not in visited and find_winner(graph, target, visited):
            return True
    visited.remove(source)
    return False

def ranked_pairs_winner(image_logits):
    num_voters, num_classes = image_logits.shape
    margins = defaultdict(int)

    # Perform pairwise comparisons
    for i in range(num_classes):
        for j in range(num_classes):
            if i != j:
                votes_for_i = torch.sum(image_logits[:, i] > image_logits[:, j])
                votes_for_j = num_voters - votes_for_i
                margins[(i, j)] = votes_for_i - votes_for_j

    # Sort pairs by margin of victory
    sorted_pairs = sorted(margins, key=margins.get, reverse=True)

    # Initialize graph for locking pairs
    graph = {i: set() for i in range(num_classes)}
    for pair in sorted_pairs:
        winner, loser = pair
        graph[winner].add(loser)
        visited = set()

        # Check for cycle
        if find_winner(graph, loser, visited):
            graph[winner].remove(loser)

    # Determine the winner
    for i in range(num_classes):
        if not any(i in targets for targets in graph.values()):
            return i

    # Fallback: Choose the class with the highest total votes if no clear winner is found
    total_votes = torch.sum(image_logits, axis=0)
    return torch.argmax(total_votes).item()
        
predictions = []
for image_logits in logits:
    rp_winner = ranked_pairs_winner(image_logits)
    predictions.append(integer_to_label[rp_winner])
    
rp_accuracy = balanced_accuracy_score(labels, predictions)
rp_accuracy

0.8667171717171718

In [39]:
from collections import defaultdict

def star_voting_winner(image_logits):
    num_voters, num_classes = image_logits.shape

    # Step 1: Sum the scores for each candidate
    total_scores = torch.sum(image_logits, axis=0)

    # Step 2: Find the two candidates with the highest total scores
    top_two = torch.topk(total_scores, 2).indices

    # Step 3: Runoff between the top two candidates
    first_choice_votes = torch.sum(image_logits[:, top_two[0]] > image_logits[:, top_two[1]])
    second_choice_votes = num_voters - first_choice_votes

    # Determine the winner
    if first_choice_votes > second_choice_votes:
        return top_two[0].item()
    else:
        return top_two[1].item()
        
predictions = []
for image_logits in logits:
    star_winner = ranked_pairs_winner(image_logits)
    predictions.append(integer_to_label[star_winner])
    
star_accuracy = balanced_accuracy_score(labels, predictions)
star_accuracy

0.8667171717171718